# NoteBook responsável por Montar árvores de Devisão

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint
from sklearn.ensemble import GradientBoostingClassifier

Carregamento do arquivo e faz os seguintes tratamento de dados:


*   Remove registros Nulos
*   Remove regitros Inválidos
*   Substitui valores nulos por zero
*   Cria campo de diferança.   

In [ ]:
pd_dados = pd.read_excel("/content/sample_data/TCC_Dados_Final1_ni.xlsx", sheet_name=0)     
#pd_dados = pd.read_csv(arquivo, sheet_name=0) 

print(pd_dados.describe())
        
print("\nRemovendo registros nulos do arquivo")
# dropna remove os registros faltantes    
pd_dados.dropna(axis=0, subset=['sit_cadastral'], inplace = True) 
pd_dados.dropna(axis=0, subset=['cnae3'], inplace = True)
pd_dados.dropna(axis=0, subset=['ni'], inplace = True)


#Remove registros que tem CNAE Inválido ou Não Informado
pd_dados['sit_cadastral'] = pd_dados['sit_cadastral'].astype('str')
pd_dados = pd_dados[~pd_dados['sit_cadastral'].str.contains('NULA')]

print("\nRemovendo registros inválidos do arquivo")
#Remove registros que tem CNAE Inválido ou Não Informado
pd_dados['cnae3'] = pd_dados['cnae3'].astype('str')
pd_dados = pd_dados[~pd_dados['cnae3'].str.contains('IN')]
pd_dados = pd_dados[~pd_dados['cnae3'].str.contains('NI')]

print("\nSubstitui valores nulos por Zero para os campos de Débitos")
#Coloca zero quando não tem valor, para não dá erro no calculo da diferença. Quando está nulo é por que não tem mais débito, ou seja, está zerado
pd_dados[['valorm1']] = pd_dados[['valorm1']].fillna(value = 0)
pd_dados[['valorm3']] = pd_dados[['valorm3']].fillna(value = 0)

print("\nCriando campos de comparação")
#calcula os campos de comparação para saber se o débito aumentou o diminuiu
pd_dados["Dif1"] = pd_dados.valorm1 - pd_dados.valor
pd_dados["Dif3"] = pd_dados.valorm3 - pd_dados.valor

print(pd_dados.describe())

pd_dados = pd_dados.loc[pd_dados['Dif1'] > 10]

print("\nSalvando o arquivo: {0}".format("/content/sample_data/TCC_Dados_Final1_ni_Aumentou.xlsx"))
pd_dados.to_excel("/content/sample_data/TCC_Dados_Final1_ni_Aumentou.xlsx", index=False)

Função que define se Aumentou, Diminuiu ou Manteve

In [ ]:
def encode_units(x):
  if x > 500:
    return "Aumentou_500"
  elif x > 10:
    return "Aumentou_10"
  else:
      return "Não Aumentou"


Campos do Modelo a ser treinado

    0 - ni	
    1 - classe	
    2 - sit_cadastral	
    3 - diferenciada	
    4 - simei	
    5 - simples	
    6 - porte	
    7 - nat_juridica	
    8 - cnae3	
    9 - valor
    10 - valorm1	
    11 - valorm3	
    12 - Dif1
    13 - Dif3

Treinamendo de Modelo sem Particionar a árvore e com Hiperparâmetros pre-definidos

In [ ]:
pd_debitos = pd.read_excel("/content/sample_data/TCC_Dados_Final1_ni_Aumentou.xlsx", sheet_name=0) 
#print("\nDimensões: {0}".format(pd_debitos.shape))

#Tranforma os valores de diferença usando o método encode_units
pd_debitos["Dif1"] = pd_debitos["Dif1"].apply(encode_units)
pd_debitos["Dif3"] = pd_debitos["Dif3"].apply(encode_units)

#apaga os valores sem diferença, manteve o débito igual
pd_debitos.dropna(axis=0, subset=['Dif1'], inplace = True)
pd_debitos.dropna(axis=0, subset=['Dif3'], inplace = True)

#print("\nQuantidade de Registros depois: {0}".format(pd_debitos["classe"].count()))

arr_atributos =['classe','sit_cadastral', 'diferenciada', 'simei','simples', 'porte', 'nat_juridica', 'cnae3'] 
le = LabelEncoder() 
#print(pd_debitos.keys())
for atributo in arr_atributos:
    pd_debitos.sort_values(by=atributo, inplace = True)
    pd_debitos[atributo] = le.fit_transform(pd_debitos[atributo]) 
    print(pd_debitos[atributo].sort_values().unique().tolist())
    print(le.inverse_transform(pd_debitos[atributo].sort_values().unique().tolist()))


#define a coluna de resultado
y_dados_m1 = le.fit_transform(pd_debitos.iloc[:,12:13])
y_dados_m3 = le.fit_transform(pd_debitos.iloc[:,13:14])
arr_label = le.inverse_transform([0,1,2])
print(le.inverse_transform([0,1,2]))
#print(y_train)    

X_dados = pd_debitos.iloc[:,0:9]

#for criterios_decisao in [['gini',10, 50], ['gini',20, 50],['gini', 20, 100],['entropy',10, 50], ['entropy',20, 50],['entropy', 20, 100]] :
'''
criterios_decisao = ['gini',10, 50]
#Cria o modelo com os HyperParametros configurados, nesse caso utiliza os Defaults

debitos_tree = DecisionTreeClassifier(random_state=0, criterion=criterios_decisao[0], min_samples_leaf=criterios_decisao[1], min_samples_split=criterios_decisao[2])
#debitos_tree = DecisionTreeClassifier(random_state=0)
#print("DecisionTreeClassifier")
debitos_tree = debitos_tree.fit(X_dados, y_dados_m3) #comando que irá treinar é o FIT
#print("Acurácia", debitos_tree.score(X_train, y_train)) #método Scrore calcula a Acurária da base de Treinamento

Train_predict = debitos_tree.predict(X_dados) 

print("Acurancia M3: {0}, criterion: {1}, min_samples_leaf: {2}, min_samples_split: {3}".format(accuracy_score(y_dados_m3, Train_predict), debitos_tree.criterion, debitos_tree.min_samples_leaf, debitos_tree.min_samples_split))
print(classification_report(y_dados_m3, Train_predict, target_names=arr_label))

#Matriz de Confusão
cnf_matrix = confusion_matrix(y_dados_m3, Train_predict)
cnf_table = pd.DataFrame(data=cnf_matrix, index=arr_label, columns=[arr_label[0] + " (prev)", arr_label[1] + " (prev)", arr_label[2] + " (prev)"])
print(cnf_table)

import pydotplus 
# Create DOT data
tree.export_graphviz(debitos_tree, out_file="/content/sample_data/TCC_Dados_Final1_ni_Aumentou.dot", 
                                proportion=False,
                                rounded =True,
                                filled=True,
                                feature_names=X_dados.keys(),
                                class_names=arr_label)
'''

Depois de criado o arquivo DOT com as informações da Árvore, cria a Imagem

In [ ]:
import pydotplus 
from IPython.display import Image


# Draw graph
graph = pydotplus.graph_from_dot_file("/content/sample_data/TCC_Dados_Final1_ni_Aumentou.dot")

# Show graph
Image(graph.create_png())